In [1]:
from __future__ import print_function
import torch
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import os
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

In [2]:
class CurrencyDataset(Dataset):
    def __init__(self,root_dir):
        self.dir = os.path.abspath(root_dir)
    def __getitem__(self,idx):
        data_path = os.path.join(self.dir,'Batches','Batch_' + str(idx) + '.npy')
        label_path = os.path.join(self.dir,'Labels','Label_' + str(idx) + '.npy')
        data = np.load(data_path)
        labels = np.load(label_path)
        return (data,labels)
    def __len__(self):
        return len(os.listdir(os.path.join(self.dir,'Batches')))

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,512,(1,1))
        self.conv2 = nn.Conv2d(512,512,(2,1))
        self.conv3 = nn.Conv2d(512,1024,(3,1))
        self.conv4 = nn.Conv2d(1024,1024,(8,1))
        self.conv5 = nn.Conv2d(1024,1024,(3,1))
        self.conv6 = nn.Conv2d(1024,1024,(3,1))
        self.conv7 = nn.Conv2d(1024,1024,(3,1))
        self.conv8 = nn.Conv2d(1024,10000,(1,1))
        self.conv9 = nn.Conv2d(10000,1,(1,1))
        self.pool = nn.AvgPool2d((3,1))
        self.norm_512 = nn.BatchNorm2d(512)
        self.norm_1024 = nn.BatchNorm2d(1024)
        self.norm_10000 = nn.BatchNorm2d(10000)
#         self.conv_test = nn.Conv2d(5,128,(48,3))
#         self.conv_test2 = nn.Conv2d(128,5,(3,1))
        self.fc1 = nn.Linear(5, 5)
        self.dp1 = nn.Dropout(p=0.3)
        self.fc2 = nn.Linear(3000, 3500)
        self.fc3 = nn.Linear(3500, 5)

    def forward(self, x):
     #   x = self.dp1(x)
#         print(x.shape)
        x = F.leaky_relu(self.conv1(x))
#         print(x.shape)
        x = F.leaky_relu(self.conv2(x))
#         print(x.shape)
        x = self.pool(x)
#         print(x.shape)
        x = F.leaky_relu(self.norm_1024(self.conv3(x)))
#         print(x.shape)
        x = F.leaky_relu(self.norm_1024(self.conv4(x)))
#         print(x.shape)
        x = F.leaky_relu(self.norm_1024(self.conv5(x)))
#         print(x.shape)
        x = F.leaky_relu(self.norm_1024(self.conv6(x)))
#         print(x.shape)
        x = F.leaky_relu(self.norm_1024(self.conv7(x)))
#         print(x.shape)
        x = F.leaky_relu(self.norm_10000(self.conv8(x)))
        x = self.conv9(x)
#         print(x.shape)
#         x = F.relu(self.conv_test(x))
#         x = self.conv_test2(x)
        
#         print(x.shape)
#         print(x.shape)
#         x = x.view(-1,5)
#         print(x.shape)
#         x = x.view(-1,5)
#         print(x.shape)       
#         x = F.relu(self.fc1(x))
#         print(x.shape)
# #         x = self.dp1(x)
#         x = F.relu(self.fc2(x))
# #         x = self.dp1(x)
#         x = self.fc3(x)
        x = x.view(-1,5)
#         x = self.dp1(x)
#         print(x.shape)
        
    
        x = F.softmax(x,dim=1)
        return x
def weight_reset(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor')
data = CurrencyDataset('./Processed')
device = 'cuda:0'
currency_count = 5
net = Net()
net = net.double()
net = net.to(device)
net.train()
# dataloader = DataLoader(data,batch_size=32,pin_memory=True, sampler=SubsetRandomSampler(list(range(512))))
dataloader = DataLoader(data,batch_size=128,pin_memory=True,shuffle=True)

In [4]:
optimizer = optim.Adam(net.parameters(), lr=0.01)

In [5]:
net.apply(weight_reset)
# test_X, test_y = iter(dataloader).next()
# print(test_X)
# print(test_y)
print_every = 16
for epoch in range(30):
    epoch_loss = 0.0
    running_loss = 0.0
    j = 0
    for i,batch in enumerate(dataloader, 0):
        j = i
        optimizer.zero_grad()
        d,y = batch
#         d = test_X
#         y = test_y
        d = d.to(device)
        x = net(d)
        y = (y.reshape([-1,5])).to(device)
#         print(x.shape)
#         print(y.shape)
        z = torch.mul(x,y)
        z = z.sum(dim=1)
        loss = z.sum()
        loss /= z.shape[0]
        epoch_loss += loss.item() * 10000
        running_loss += loss.item() * 10000
        loss = loss ** -1
        
        if i % print_every == print_every - 1:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / print_every))
            running_loss = 0.0

        loss.backward()
        optimizer.step()
    print(epoch_loss/(j+1))

[1,    16] loss: 9999.931
[1,    32] loss: 9999.918
[1,    48] loss: 9999.875
[1,    64] loss: 9999.876
[1,    80] loss: 9999.836
[1,    96] loss: 9999.919
[1,   112] loss: 9999.787
[1,   128] loss: 9999.831
[1,   144] loss: 9999.921
[1,   160] loss: 9999.822
[1,   176] loss: 9999.928
[1,   192] loss: 9999.875
[1,   208] loss: 9999.909
[1,   224] loss: 9999.954
[1,   240] loss: 9999.970
[1,   256] loss: 9999.838
[1,   272] loss: 9999.964
[1,   288] loss: 9999.925
[1,   304] loss: 10000.026
[1,   320] loss: 9999.998
[1,   336] loss: 9999.940
[1,   352] loss: 9999.970
[1,   368] loss: 9999.967
[1,   384] loss: 10000.005
[1,   400] loss: 9999.991
[1,   416] loss: 9999.999
[1,   432] loss: 9999.998
[1,   448] loss: 10000.009
[1,   464] loss: 10000.015
[1,   480] loss: 10000.020
[1,   496] loss: 9999.932
[1,   512] loss: 9999.939
[1,   528] loss: 9999.977
[1,   544] loss: 9999.930
[1,   560] loss: 10000.007
[1,   576] loss: 9999.984
[1,   592] loss: 10000.010
[1,   608] loss: 9999.997
[1,  

[2,   944] loss: 10000.005
[2,   960] loss: 10000.005
[2,   976] loss: 10000.006
[2,   992] loss: 10000.032
[2,  1008] loss: 9999.981
[2,  1024] loss: 9999.967
[2,  1040] loss: 10000.029
[2,  1056] loss: 9999.933
[2,  1072] loss: 9999.990
[2,  1088] loss: 9999.950
[2,  1104] loss: 10000.007
[2,  1120] loss: 9999.996
[2,  1136] loss: 9999.995
[2,  1152] loss: 10000.004
[2,  1168] loss: 9999.985
[2,  1184] loss: 10000.010
[2,  1200] loss: 9999.976
[2,  1216] loss: 9999.944
[2,  1232] loss: 10000.036
[2,  1248] loss: 9999.958
[2,  1264] loss: 9999.968
[2,  1280] loss: 9999.946
[2,  1296] loss: 10000.023
[2,  1312] loss: 9999.983
[2,  1328] loss: 10000.012
[2,  1344] loss: 10000.001
[2,  1360] loss: 10000.026
[2,  1376] loss: 9999.965
[2,  1392] loss: 10000.067
[2,  1408] loss: 9999.955
[2,  1424] loss: 10000.023
[2,  1440] loss: 9999.975
[2,  1456] loss: 9999.989
[2,  1472] loss: 9999.973
[2,  1488] loss: 9999.992
[2,  1504] loss: 9999.972
[2,  1520] loss: 10000.009
[2,  1536] loss: 9999.

[3,  1872] loss: 9999.853
[3,  1888] loss: 9999.907
[3,  1904] loss: 9999.858
[3,  1920] loss: 9999.943
[3,  1936] loss: 9999.849
[3,  1952] loss: 9999.948
[3,  1968] loss: 9999.882
[3,  1984] loss: 9999.944
[3,  2000] loss: 9999.229
[3,  2016] loss: 9999.877
[3,  2032] loss: 9999.858
[3,  2048] loss: 9999.864
[3,  2064] loss: 9999.893
[3,  2080] loss: 9999.891
[3,  2096] loss: 9999.906
[3,  2112] loss: 9999.863
[3,  2128] loss: 9999.924
[3,  2144] loss: 9999.990
[3,  2160] loss: 9999.909
[3,  2176] loss: 9999.784
[3,  2192] loss: 9999.999
[3,  2208] loss: 9999.800
[3,  2224] loss: 9999.868
[3,  2240] loss: 9999.883
[3,  2256] loss: 9999.840
[3,  2272] loss: 9999.870
[3,  2288] loss: 9999.853
[3,  2304] loss: 9999.924
[3,  2320] loss: 10000.926
[3,  2336] loss: 9999.863
[3,  2352] loss: 9999.875
[3,  2368] loss: 9999.892
[3,  2384] loss: 9999.907
[3,  2400] loss: 9999.810
[3,  2416] loss: 9999.912
[3,  2432] loss: 9999.874
[3,  2448] loss: 9999.899
[3,  2464] loss: 9999.971
[3,  2480] 

[4,  2832] loss: 9998.214
[4,  2848] loss: 9999.874
[4,  2864] loss: 9999.878
[4,  2880] loss: 9999.804
[4,  2896] loss: 9999.736
[4,  2912] loss: 9999.952
[4,  2928] loss: 9999.897
[4,  2944] loss: 9999.892
[4,  2960] loss: 9999.837
[4,  2976] loss: 9999.851
[4,  2992] loss: 9999.892
[4,  3008] loss: 9999.862
[4,  3024] loss: 9999.946
[4,  3040] loss: 9999.829
[4,  3056] loss: 9999.951
[4,  3072] loss: 9999.847
[4,  3088] loss: 9999.904
[4,  3104] loss: 10000.010
[4,  3120] loss: 9999.935
[4,  3136] loss: 9999.859
[4,  3152] loss: 9999.869
[4,  3168] loss: 9999.808
[4,  3184] loss: 9999.718
[4,  3200] loss: 9999.775
[4,  3216] loss: 9999.820
[4,  3232] loss: 9999.993
[4,  3248] loss: 9999.872
[4,  3264] loss: 9999.909
[4,  3280] loss: 9999.956
[4,  3296] loss: 9999.897
[4,  3312] loss: 9999.901
[4,  3328] loss: 9999.813
[4,  3344] loss: 9999.904
[4,  3360] loss: 9999.944
[4,  3376] loss: 9999.788
[4,  3392] loss: 9999.877
[4,  3408] loss: 9999.945
[4,  3424] loss: 9995.018
[4,  3440] 

KeyboardInterrupt: 

In [ ]:
# net.eval()
# for i,batch in enumerate(dataloader):
#     d,y = batch
#     print(y)
#     print(net(d.cuda()))
#     break